# Chapter 7: Form Mason – a Monkey of your own


In our journey to learn Django, we have traveled far. We started from baby steps,
learning to set up databases, basic views, and templates. We then moved on to more
difficult stuff such as management commands and the Django shell. This is the last
chapter in that journey, where we use all the knowledge that we have gained, and
use it to create one of the most complicated applications yet. It's always fun to end
things with a bang, and that's what we'll do here!

You might have heard of SurveyMonkey (<www.surveymonkey.com>) or Wufoo
(<www.wufoo.com>). If not, these are web applications that allow you to create custom
forms to gather data from audiences. You use the control panel available on these
sites to set up a form, defining all the fields and how they should be validated, and
configuring some basic stuff such as what the form should look like, what theme it
should use, and so on. After configuring the form, you get a link to a web page that
displays that form.

You then send this link out to the intended audience and they fill it in, with their
responses being saved. As part of your control panel, you also get a web page where
you can view and filter these responses.

Services like these make it dead simple for even the least computer-savvy people to
create online surveys and gather data for any purpose from a wide audience. This
seems like a cool service that we will replicate in this chapter. It will also be the most
complex application that we will have created, as it requires us to dive deep into
Django and understand how Django forms work behind the scenes. In a sense, you
will learn about the magic that makes Django tick. Exciting, isn't it!

## Code pack

This chapter does not have a code pack because almost all of the stuff that we are
about to do here is new. Plus, we will do a lot of exploring in this chapter as well.
This is both fun and shows you one of the many ways of approaching a tough project.

As there is no code pack to work from, you'll need to start the project by yourself.
First, as always, create a new virtual environment for the new project. Next, with
the environment activated, install Django and run the following:

```sh
> django-admin.py startproject formmason
```

This should create a `formmason` folder with our new Django project, ready for us
to start working on. Use the cd command to get into this folder and create a new
application that we'll be using to hold our views, models, and so on:

```sh
python manage.py startapp main
```

Finally, add the main application to our list of `INSTALLED_APPS` in `formmason/settings.py`. Once that's done, we're ready to start!

## Looking at a Django form

As our aim is to allow users to create dynamic forms based on parameters stored in a
database, a good place to start is to look at how a Django form works under the hood
and what options we have to customize it. First, let's create a basic form. Create a
new `main/forms.py` file and add the following code to it:

```python
from django import forms

class SampleForm(forms.Form):
    name = forms.CharField()
    age = forms.IntegerField()
    address = forms.CharField(required=False)
    gender = forms.ChoiceField(choices=(('M', 'Male'), ('F', 'Female')))
```

This is a pretty basic form. However, it has a variety of form fields that we can look
at. Let's play around a bit in the shell, which you can start as follows:

```sh
> python manage.py shell
```

> tips

> I usually like to install another package called ipython. When you
start the Django shell with ipython installed, you get an enhanced
version of the basic shell with a lot of cool features such as
autocomplete and a much better looking interface. I always install
it in any Django project because I almost always use the shell at the
start of the project to play around. I highly advise you to install it as
well when you start a new Django project.

Import our form in the shell as follows:

```python
> from main.forms import SampleForm
```

In the same shell, type the following:

```python
> form = SampleForm()
> form.fields
OrderedDict([('name', <django.forms.fields.CharField at 0x10fc79510>),
('age', <django.forms.fields.IntegerField at 0x10fc79490>),
('address', <django.forms.fields.CharField at 0x10fc79090>),
('gender', <django.forms.fields.ChoiceField at 0x10fc792d0>)])
```
            

The first line of code simply created an instance of our form class. It's the second line
where the fun begins. However, I'll first explain the OrderedDict data structure
a bit.

As the name suggests, `OrderedDict` is a dictionary that maintains the order in which
its elements were inserted. A normal dictionary in Python has no fixed order. If you
insert three elements into a dictionary with keys A, B, and C, and then you ask for the
keys back using the `keys()` method on the dictionary instance, the order in which
you will get them back is not guaranteed, which is why normal built-in dictionaries
are said to be unordered.

In contrast, the keys in `OrderedDict`, which is from the `collections` library (part
of the Python standard library), are guaranteed to be in the same order that you
inserted them in. So if you were to iterate over the keys using either the keys()
or `items()` method, you would always get them back in the same order that you
inserted them.

Getting back to the output, you will see that the dictionary printed has the same keys
as the names of the fields that we used when we created our `SampleForm` class. The
values of these keys are the `Field` subclasses (`CharField`, `IntegerField`, and so on)
that we used in our form.

Let's try something. In the shell, type the following and look at the output:

```python
> form.name
---------------------------------------------------------------------
------
AttributeError Traceback (most recent call
last)
<ipython-input-16-1174e5d9164a> in <module>()
----> 1 form.name
AttributeError: 'SampleForm' object has no attribute 'name'
```

It seems that the `name` attribute that we defined on our `SampleForm` class is no longer
there. Weird, huh?

So far you have learned two facts about a Django form. The first, that the fields
attribute on a form instance contains a mapping of field names to the `Field` classes.
Second, that the field attributes we defined when creating our `SampleForm` class are
not accessible on the instance

Putting these two facts together gives us some indication of what Django does with
the `SampleForm` class when creating an instance of it. It removes the `field` attributes
and adds them to the `fields` dictionary. After a form instance is created, the only
way to find out the fields that are defined on it is the `fields` dictionary attribute.

So, if we wanted to add another field to a form after the class has been defined
and we could not change the code of the class itself, we could add an instance of a
`Field` subclass to the `fields` attribute on the form instance. Let's try it out and see
if this works.

> tips

> I lied a little when I said that Django removes the field attributes
from a `SampleForm` class when creating an instance of it. In reality,
the `SampleForm` class also has its field attributes removed. If you
typed `SampleForm.name` in the shell, you would get a similar
AttributeError. However, this information is not relevant to our
current task, and the reason why this happens is complicated so I
won't go into it in this book. If you want all the details, check out the
source for the django.forms.Form class.

### Adding an extra field to a SampleForm instance

We will now try an experiment. We will create an instance of our `SampleForm` with
some valid test data, and then we will add another field to the `fields` attribute on
our instance. We will then call the `is_valid()` method to see if our form instance
considers the dynamically added field when validating the data provided earlier.
Let's see what happens. In the Django shell, enter the following commands

```python
> from main.forms import SampleForm
> test_data = {'name': 'Jibran', 'age': 27, 'gender': 'M'}
> form = SampleForm(test_data)
> from django import forms
> form.fields['country'] = forms.CharField()
> form.is_valid()
False
> form.errors
{'country': [u'This field is required.']}
```

Nice! It seems like our idea worked. Even though we added the `country` field after
our` SampleForm` had been instantiated and provided the data, the form validation
did consider our new field. As the `country` field was not part of the `test_data`
dictionary and the field is required, the form validation failed and the `errors` list
included the appropriate error.

Now that we have a technique to achieve our objective, let's create some views and
templates to render a dynamic form.

## Generating dynamic forms 

Before we get to the code, we have to decide on a data format to specify the structure
of our dynamic forms. **JSON** is one of the most popular, if not the most popular, data
storage and exchange formats on the web right now. You will probably be aware of
what JSON, is and how it works and what it looks like. However, if you're not, here's
a quick introduction.

JSON stands for JavaScript Object Notation. It uses the same format that JavaScript
uses to declare objects. The biggest benefit for us Python guys is that it is almost
exactly the same as the dictionary declaration syntax in Python. Let's say that we
want to store some details about a user. In JSON, here is what that information
would look like:

```json
{
    "name": "Jibran",
    "age": 27,
    "country": "Pakistan"
}
```

As you can see, we can even copy and paste this into our Python code and it would
be a valid dictionary definition, which makes it very easy for us to work with.

JSON has a number of benefits over other data storage/exchange formats:
* It's text only, so no need to have special tools built around it for the viewing
and parsing.
* Parsing JSON is very easy, and most languages have a library for this. Python
comes with a standard library to parse JSON.
* It's simple to write by hand.
* It can be stored in a number of ways without needing to do anything special.
We will use this fact later on to store JSON in our database as a simple text
field.

Now that you know a bit about JSON, let's see how to use it. We will store
information about how to construct a dynamic form in JSON. We will then use the
Python standard library `json` to convert the JSON string to a Python dictionary,
which we will then iterate over and create our form.

> tips

> For this section, we will hardcode the JSON that we use in order to
generate the form. Later on, we will allow our users to create and
store the JSON themselves in a database from a control panel that
we will make for them.

We will look at how to define the form fields in JSON and generate a dynamic form
from this schema. We will also create an HTML form to render our form so we can
test that everything works as expected.

Let's look at the format for a simple form that can be used to collect basic
demographic information about people:

```json
{
    "name": "string",
    "age": "number",
    "city": "string",
    "country": "string",
    "time_lived_in_current_city": "string"
}
```

This is the sample JSON that we will use in this section to generate and display
our dynamic form. This structure is a simple dictionary with strings for the values.
We have to write code to parse this dictionary and create a Django form out of this
information.

### Generating a form out of JSON

The first thing that we will need to do is convert the JSON data that our view will
get to a Python dictionary so that we can iterate over it. Python comes with the json
module as part of the standard library that will handle the parsing for us. You can
read the documentation for it at <https://docs.python.org/3/library/json.html>. It is dead simple to use, however, so let's just dive in.

Open up `main/views.py` and add this code:

```python
import json
from django import forms
from django.views.generic import FormView

class CustomFormView(FormView):
    template_name = "custom_form.html"

    def get_form(self):
        form_structure_json = """{
            "name": "string",
            "age": "number",
            "city": "string",
            "country": "string",
            "time_lived_in_current_city": "string"}
            """

        form_structure = json.loads(form_structure_json)
        custom_form = forms.Form(**self.get_form_kwargs())
        
        for key, value in form_structure.items():
        field_class = self.get_field_class_from_type(value)
        if field_class is not None:
            custom_form.fields[key] = field_class()
        else:
            raise TypeError("Invalid field type {}".format(value))
        
        return custom_form

    def get_field_class_from_type(self, value_type):
        if value_type == "string":
            return forms.CharField
        elif value_type == "number":
            return forms.IntegerField
        else:
            return None
```

There are a couple of things to explain here. However, let's first get the view working
and see if it does what we want. Then I'll explain what this code does. Next, let's
create the `main/templates/custom_form.html` template. You will need to create
the `templates` folder under the `main/` folder first. Put this code in there:

```html
<html>
<head>
    <meta http-equiv="content-type" content="text/html; charset=utf-8"
    />
    <title>Custom Form Demo</title>
</head>
<body>
    <h1>Custom Form</h1>
    <form action="" method="post">{% csrf_token %}
        {{ form.as_p }}
        <input type="submit" value="Submit" />
    </form>
</body>
</html>
```

Finally, put the following code in `formmason/urls.py` to include our new view:

```python
from django.conf.urls import url
from main.views import CustomFormView

urlpatterns = [
        url(r'^$', CustomFormView.as_view(), name='custom-form'),
    ]
```

That's it for now. To test it out, run the development server using the runserver
command and then open up `http://127.0.0.1:8000` in any browser. You should
see a screen similar to this:

![Form Custom](img/Mason_CustomForm.png)

Try submitting an incomplete form and the page should be rendered with the correct
error messages. Try to complete the form and submit it. Now you should see an
error page, with the ImproperlyConfigured: *No URL to redirect to. Provide
a success_url* error. This error comes from our use of the `FormView` generic view.
When the form is valid, it expects `success_url` to be defined so that the user can be
redirected to it. We don't have anywhere to redirect the user to, so we will ignore this
for now. Let's take a look at the code and see what we have done here.

>tips

> This example also highlighted the power of generic views. By just
redefining one function, get_form, we were able to use all the
features of FormView with a form that is generated dynamically.
As the Django generic views are made with modularity in mind,
the rest of the view does not care how the form is created, just that
the get_form method returns a valid form.

Most of the action happens in the `get_form` method of our CustomFormView, so let's
start there. The first line of code defines the JSON data structure that we will use to
generate our form. As I've mentioned before, this data will eventually be coming
from a database backend; however, for testing purposes, we are hardcoding it for
now. Next, we use the `json.loads` method to convert the JSON string to a Python
object. This conversion process is pretty intuitive. The JSON string that we have is a
dictionary with keys for name, age, city, country, and `time_lived_in_current_city`. The Python object that we get from `json.loads` is a dictionary as well, with
the same key/value pairs. You can also have arrays in your JSON string and loading
it would give you a Python list. JSON has support for nested objects, so you could
have an array of dictionaries or a dictionary with an array for a value.

Next, we create an instance of the base `django.forms.Form` class. As we saw in our
experimentation before, we can take an instance of a Django form and add fields to
it. Instead of creating an empty form class and using that, we just use the base Form
class from Django directly. To the constructor of the form class, we pass whatever we
receive from `self.get_form_kwargs()`. This method is part of the Django `FormView`
class and depending on the request type creates the correct keyword arguments to
be passed to a Form class. For instance, if the request was a POST/PUT request, `get_form_kwargs()` would return a dictionary that contains the `data` keyword argument
so that we can use the form to validate and take further action on the data.

The interesting bit happens next, when we loop over the items list of our custom
fields data. The `items()` method on the data that we loaded from JSON returns a list
as follows:

```python
[('name', 'string'), ('age', 'number'), ('city', 'string'),
('country', 'string'), ('time_lived_in_city', 'string')]
```

We loop over this list, assigning each of these item pairs to variables key and value
in the `for` loop. We then pass the values to the `get_field_class_from_type`
method, which decides which of the available form field classes to use for the type
of data passed. The method's code is a simple if/else, which returns a field class or
`None` if it was passed an invalid type.

We use the return value of this method, which is a class, and assign an instance of
it to the form's `fields` attribute dictionary. *Note that we assign an instance of the field class and not the class itself*. The name of the field is the key from our JSON dictionary.
We also do some basic error handling, raising `TypeError` if we cannot find a
matching field class for the type we got in our JSON data dictionary.

Finally, we return our customized form class. From there on, the Django FormView
takes over and either renders the form, with errors if necessary, or redirects the user
to the success URL if the form data submitted by the user was valid. As we didn't
define any success URL, we get an error when we submit a valid form.

That's it. That is all the code that we need to create a dynamically generated form.
We can add a couple of more features such as custom validation or advanced
features such as supporting choice fields with a limited set of choices, but I'll
leave that as an interesting project for you to try on your own.

That's it. That is all the code that we need to create a dynamically generated form.
We can add a couple of more features such as custom validation or advanced
features such as supporting choice fields with a limited set of choices, but I'll
leave that as an interesting project for you to try on your own.

Next, let's take a look at how we can store the JSON data that defines our custom
form in a database. We'll create a model for it and allow the user to create multiple
forms with different fields from an admin panel.

## A model for our JSON

As I have mentioned earlier, one of the biggest benefits of using JSON as the
definition format for our forms is that it uses just simple text data to encode the
definition of complex objects. While some databases such as PostgreSQL have a
column type for JSON, others do not. However, because we are dealing with simple
text data, we don't need one! We can store our JSON data in a simple `TextField`
and then encode and decode the data to and from a Python dictionary whenever
required. In fact, there are many people in the Django community who have already
dealt with this problem and open sourced their solutions for us to use.

One such package that I have used in the past is django-jsonfield. You can find it
at <https://github.com/bradjasper/django-jsonfield>, and we will be using it
in our project. First, install the required package by typing the following command in
your command line. Make sure to have the virtual environment activated first so that
it is installed in the correct location.

```sh
> pip install jsonfield
```

With the package installed, we can create a model for our form. Open up `main/models.py` and change it to have the following code:

```python
from __future__ import unicode_literals
from django.db import models
from jsonfield import JSONField

class FormSchema(models.Model):
    title = models.CharField(max_length=100)
    schema = JSONField()
    
```

Save the file and then create and run migrations so that Django creates the tables for
our new model. In the command line, run the following commands:

```
> python manage.py makemigrations main
Migrations for 'main':
0001_initial.py:
- Create model FormSchema
> python manage.py migrate
Operations to perform:
Apply all migrations: sessions, contenttypes, admin, main, auth
Running migrations:
Rendering model states... DONE
Applying contenttypes.0001_initial... OK
Applying auth.0001_initial... OK
Applying admin.0001_initial... OK
Applying admin.0002_logentry_remove_auto_add... OK
Applying contenttypes.0002_remove_content_type_name... OK
Applying auth.0002_alter_permission_name_max_length... OK
Applying auth.0003_alter_user_email_max_length... OK
Applying auth.0004_alter_user_username_opts... OK
Applying auth.0005_alter_user_last_login_null... OK
Applying auth.0006_require_contenttypes_0002... OK
Applying auth.0007_alter_validators_add_error_messages... OK
Applying sessions.0001_initial... OK
```

With the migrations done, let's see how we can use this model. Instead of writing the
code for the view, I like to start by doing a bit of experimentation in the Django shell.
Open it up by typing as follows:

```sh
> python manage.py shell
```

Then type in the following to test out our new model:

```python
> from main.models import FormSchema
> fs = FormSchema()
> fs.title = 'My First Form'
> fs.schema = {'name': 'string', 'age': 'number', 'city': 'string',
'country': 'string', 'time_lived_in_current_city': 'string'}
> fs.save()
> FormSchema.objects.get(pk=1).schema
{u'age': u'number',
 u'city': u'string',
 u'country': u'string',
 u'name': u'string',
 u'time_lived_in_current_city': u'string'}
```

What we have done here should be pretty simple for you to understand by now.
The important thing to note here is the value that we assigned to the schema field.
Instead of using a string representation of the JSON data, we simply assigned a
Python dictionary to it. The `JSONField` class we used as the field class in our model
does the heavy lifting of converting from a Python dictionary to a JSON string when
saving to the database.

The reverse is also the same. Notice how, in the last line of our shell session, we
simply accessed the schema field directly and got a Python dictionary back instead of
the string JSON data that is actually saved in the database. This makes working with
JSON transparent to us.

> tips

> You might be wondering why I am asking you to experiment in this
chapter and play around in the shell, whereas in the previous chapters, I
just showed you the relevant view/model/template code directly.
Like I mentioned at the start of this chapter, this chapter is about showing
you how to arrive at a solution by yourself instead of me holding your
hand and showing you the end result immediately.



> All good Django developers that I know have a similar method of
developing solutions for the projects that they work on. They experiment
a bit, and in doing so figure out a solution to the problem they are
working on. Everyone has a different way of experimenting though. Some
people, like me, use the Django shell a lot. Others write test code in views
and models. Others might create simple Django management commands
to do the same. However, everyone goes through the same process.

> We find a problem that needs to be solved, we research it a bit, and then
experiment with the various methods of solving it, finally choosing one
which we like the best. You will eventually develop a method that you
are comfortable with. In the meanwhile, I'll be walking you through my
method and you can use this if you like.

We now have a FormSchema object in our database, so let's create a view that can use
this object to generate a form. In `main/views.py`, first import our new model near
the top:

```python
from main.models import FormSchema
```

Then change the `get_form` method in our `CustomFormView` to match this:

```python
def get_form(self):
    form_structure = FormSchema.objects.get(pk=1).schema
    
    custom_form = forms.Form(**self.get_form_kwargs())
    for key, value in form_structure.items():
        field_class = self.get_field_class_from_type(value)
        if field_class is not None:
            custom_form.fields[key] = field_class()
        else:
            raise TypeError("Invalid field type {}".format(value))
    
    return custom_form
```

I have highlighted the new line. We have removed the hardcoded JSON string that
we used and instead assigned the `schema` field's value from our database object to
the `form_structure` variable. The rest of the code stays the same. Try opening the
home page for the application again. You'll find that the frontend has stayed the
same. The interactions will be the same as well. You can try to submit invalid or
incomplete data and it will show you the errors as before. Trying to submit a valid
form will still result in the error about success URL not being defined.

Next, let's create a better frontend for our users. We'll create a list view where the
user can see a list of all forms available on the site and a form view that displays the
actual form and handles the interactions.

## Creating a better user interface

There isn't anything complicated about what we'll do here. It should be very easy for
you to follow along, so I'll just give you the code to write and leave the explanation,
as we have done all of this many times before in the previous chapters.

Start by creating a `templates` directory in the project root. Next, add it to the list of
`DIRS` in the `TEMPLATES` configuration variable in our `formmason/settings.py` file.
The `settings.py` file already has a `TEMPLATES` variable configured, so go ahead and
replace the `DIRS` list in this dictionary with the value that you see here:

```python
TEMPLATES = [
    { 
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': [
            os.path.join(BASE_DIR, 'templates'),
        ],
        'APP_DIRS': True,
        'OPTIONS': {
            'context_processors': [
                'django.template.context_processors.debug',
                'django.template.context_processors.request',
                'django.contrib.auth.context_processors.auth',
                'django.contrib.messages.context_processors.messages',
            ],
        },
    },
]
```

Next, create a `base.html` template in the new `templates` directory that you just
created and put this code in there:

```html
<html>
<head>
    <meta http-equiv="content-type" content="text/html; charset=utf-8"
    />
    <title>Form Mason</title>
</head>
<body>
    <a href="{% url 'home' %}">Home</a>
    {% block content %}
    {% endblock %}
</body>
</html>
```

Modify `main/templates/custom_form.html` to match this:

```html
{% extends "base.html" %}

{% block content %}
    <h1>Custom Form</h1>
    <form action="" method="post">{% csrf_token %}
        {{ form.as_p }}
        <input type="submit" value="Submit" />
    </form>
{% endblock %}
```

Change `main/views.py` to this:

```python
from django import forms
from django.views.generic import FormView
from django.views.generic import ListView
from main.models import FormSchema


class HomePageView(ListView):
    model = FormSchema
    template_name = "home.html"


class CustomFormView(FormView):
    template_name = "custom_form.html"

    def get_form(self):
        form_structure = FormSchema.objects.get(pk=self.kwargs["form_pk"]).schema
        
        custom_form = forms.Form(**self.get_form_kwargs())
        for key, value in form_structure.items():
            field_class = self.get_field_class_from_type(value)
            if field_class is not None:
                custom_form.fields[key] = field_class()
            else:
                raise TypeError("Invalid field type {}".format(value))
        
        return custom_form

    def get_field_class_from_type(self, value_type):
        if value_type == "string":
            return forms.CharField
        elif value_type == "number":
            return forms.IntegerField
        else:
            return None
```

Create a new home template at `main/templates/home.html` and give it the
following code:

```html
{% extends "base.html" %}

{% block content %}
    <h1>Available Forms</h1>
    {% if object_list %}
    <ul>
        {% for form in object_list %}
        <li><a href="{% url 'custom-form' form_pk=form.pk %}">{{ form.
        title }}</a></li>
        {% endfor %}
    </ul>
    {% endif %}
{% endblock %}
```

Finally, change `formmason/urls.py` to match this:

```python
from django.conf.urls import url
from main.views import CustomFormView
from main.views import HomePageView

urlpatterns = [
    url(r'^$', HomePageView.as_view(), name='home'),
    url(r'^form/(?P<form_pk>\d+)/$', CustomFormView.as_view(), name='custom-form'),
]
```

Once you have done all this, open up the home page for the application again at
`http://127.0.0.1:8000`, and you should see a page similar to this:

![Form Masion: Available Forms](img/Mason_Forms.png)

that we had before; the only difference is that now it is served at the URL,
`http://127.0.0.1:8000/form/1/`.

Like I said, all this is pretty basic stuff that we have done repeatedly in the past few
chapters. One thing that might be new is our use of `self.kwargs['form_pk']` in the
`CustomFormView.get_form` method. Here's the relevant line:

```python
form_structure = FormSchema.objects.get(pk=self.kwargs["form_pk"]).schema
```

For any generic view that Django provides (except for the base `View` class), self.
kwargs is a dictionary of all named parameters that matched the URL pattern. If you
look at our `formmason/urls.py` file, we defined the URL for our custom form page
like this:

```python
url(r'^form/(?P<form_pk>\d+)/$', CustomFormView.as_view(), name='custom-form')
```

In our view, the `form_pk` parameter defined in the regex pattern for the URL is
available in `self.kwargs`. Likewise, any non-keyword parameters in the URL
pattern are available for use in `self.args`.

Now that we have a useable user interface, we will move on to storing the form
responses in our database and giving our customers a page to view these responses.

## Saving the responses

We want to save the response of the user filling in one of our dynamic forms. As
the data from a dynamic form will also be dynamic, we need to store an unknown
number of fields and their values in our database. As we have already seen, JSON is
a reasonable way to store such data. Let's create a new model in `main/models.py` to
store the responses:

```python
class FormResponse(models.Model):
    form = models.ForeignKey(FormSchema)
    response = JSONField()
```


Next, create and run the migrations to add this new model to our database:

```sh
> python manage.py makemigrations main
Migrations for 'main':
  0002_formresponse.py:
    - Create model FormResponse

> python manage.py migrate main
Operations to perform:
  Apply all migrations: main
Running migrations:
  Rendering model states... DONE
  Applying main.0002_formresponse... OK
```  

Now that we have our model, we need to save valid responses for our forms in this
model. The `form_valid` method of `CustomFormView` is the correct place to add this
logic. First, we need to import a few things from Django and our new model at the
top of our `main/views.py` file:

```python
from django.core.urlresolvers import reverse
from django.http.response import HttpResponseRedirect
from main.models import FormResponse
```

Then, add a `form_valid` method with this code to the `CustomFormView` class:

```python
def form_valid(self, form):
    custom_form = FormSchema.objects.get(pk=self.kwargs["form_pk"])
    user_response = form.cleaned_data

    form_response = FormResponse(form=custom_form, response=user_response)
    form_response.save()

    return HttpResponseRedirect(reverse('home'))
```

That's it. Now try submitting the custom form that we created earlier and if the data
was valid, you should be redirected to the home page after your response was saved.
Right now, we have no way to see these responses in the frontend; however, we can
use the Django shell to make sure that our data has been saved. Start the shell with
the following command:

```sh
> python manage.py shell
```

Then use the following lines of code to see the response that was saved:

```sh
> from main.models import FormResponse
> FormResponse.objects.all()[0].response
{u'age': 27,
 u'city': u'Dubai',
 u'country': u'UAE',
 u'name': u'Jibran',
 u'time_lived_in_current_city': u'3 years'}
```

You should see the data that you entered when saving the form. Now let's create a
screen where our customers can see the responses for their custom forms.

## Showing the responses

The code in this section is pretty simple and should not have any surprises for you.
First, let's create the view in `main/views.py`:

```python
class FormResponsesListView(ListView):
    template_name = "form_responses.html"

    def get_context_data(self, **kwargs):
        ctx = super(FormResponsesListView, self).get_context_data(**kwargs)
        ctx["form"] = self.get_form()

        return ctx

    def get_queryset(self):
        form = self.get_form()
        return FormResponse.objects.filter(form=form)

    def get_form(self):
        return FormSchema.objects.get(pk=self.kwargs["form_pk"])
```        

Next, create the `main/templates/form_responses.html` template:

```html
{% extends "base.html" %}

{% block content %}
<h1>Responses for {{ form.title }}</h1>
{% if object_list %}
<ul>
    {% for response in object_list %}
    <li>{{ response.response }}</li>
    {% endfor %}
</ul>
{% endif %}
{% endblock %}
```

In `formmason/urls.py`, import our new view:

```python
from main.views import FormResponsesListView
```

Add this URL pattern to the urlpatterns list:

```python
url(r'^form/(?P<form_pk>\d+)/responses/$', FormResponsesListView.as_view(), name='form-responses'),
```

Finally, edit `main/templates/home.html` to add a link to this new view:

```html
{% extends "base.html" %}

{% block content %}
    <h1>Available Forms</h1>
    {% if object_list %}
    <ul>
        {% for form in object_list %}
        <li>
            <a href="{% url 'custom-form' form_pk=form.pk %}">{{ form.title }}</a><br />
            <a href="{% url 'form-responses' form_pk=form.pk %}">See Responses</a>
        </li>
        {% endfor %}
    </ul>
    {% endif %}
{% endblock %}
```

The new line of code is highlighted. After making all these changes, open the home
page and you should see the **See Responses** links for the new view next to each of
the existing form links:

![Form Mason: See Response](img/Mason_Responses.png)

Clicking on the link should take you to a page as follows:

![Form Mason: Responses for My First Form](img/Mason_ResponsesResult.png)

While it does get the job done, it is very crude. We can do better. Let's improve this.

### An improved responses list

What we'd like is to show the responses in a tabular fashion with the field names as
the header and response values for these fields underneath them. Let's start out by
modifying our view code. In `main/views.py`, first import `TemplateView` as we will
no longer be using `ListView` as the base class for our FormResponsesListView:

```python
from django.views.generic import TemplateView
```

Next, modify `FormResponsesListView` to match the following code:

class FormResponsesListView(TemplateView):
    template_name = "form_responses.html"

    def get_context_data(self, **kwargs):
        ctx = super(FormResponsesListView, self).get_context_data(**kwargs)

        form = self.get_form()
        schema = form.schema
        form_fields = schema.keys()
        ctx["headers"] = form_fields
        ctx["form"] = form
        
        responses = self.get_queryset()
        responses_list = list()
        for response in responses:
            response_values = list()
            response_data = response.response

            for field_name in form_fields:
                if field_name in response_data:
                    response_values.append(response_data[field_name])
            else:
                response_values.append('')
            responses_list.append(response_values)

        ctx["object_list"] = responses_list

        return ctx

    def get_queryset(self):
        form = self.get_form()
        return FormResponse.objects.filter(form=form)

    def get_form(self):
        return FormSchema.objects.get(pk=self.kwargs["form_pk"])
```        

The major changes here are in the `get_context_data` method. We also changed the
base class to `TemplateVie`w from ListView. Let's look at what we are doing in the
`get_context_data` method.

First, we use the `keys` method on the JSON form schema, which is saved in the
FormSchema model, to create a list of field headers. We pass this to the template in
the `headers` context variable.

The slightly complicated part comes next. We loop over each of the `FormResponse`
objects in our database. For each response object, we then loop over the form field
names and get that attribute from the response data. We do this because, in the
Django template, there is no way to get a value from a dictionary with a variable key
name. We could create a custom template tag; however, that would add unnecessary
complexity. Instead, we do the same thing in our view, where it is easier. For each
response object, we create a list of values that is in the same order as the field headers.
We then add this list of field values to our list of responses. The reason that our data is
structured in this way becomes clear when we look at our template.

Finally, we assign the list of responses to the `object_list` template context variable
and return the context. Next, let's change the `main/templates/form_responses.html` template to match the following:

```html
{% extends "base.html" %}

{% block content %}
<h1>Responses for {{ form.title }}</h1>
{% if object_list %}
<table border="1px">
    <tr>
        {% for field_name in headers %}
        <th>{{ field_name }}</th>
        {% endfor %}
    </tr>
    {% for response in object_list %}
    <tr>
        {% for field_value in response %}
        <td>{{ field_value }}</td>
        {% endfor %}
    </tr>
    {% endfor %}
</table>
{% endif %}
{% endblock %}
```

If you understood how our data was structured in the view, the template should be
simple enough to follow. These are all the changes that we need to make for now.

If you open the responses page again, you should now see a neatly aligned table, as
shown in the following screenshot:

![Form Mason: Responses for My First Form](img/Mason_Responses2.png)

We have so far created a page to list the available forms, a page to submit data with
a dynamic form, and a page to show those responses. Our application is nearly done.
However, we are still missing a page that allows customers to create a custom form.
Let's tackle this next.

## Designing a form creation interface

We want to provide our users with a friendly interface to edit the structure of their
dynamic forms. The ideal interface would be something that allows users to drag
and drop their fields and set the properties for these fields using simple click and
type operations on a web page. However, creating an interface like this is a major
undertaking and a project on its own. It requires a team of frontend developers and
designers to create something that user friendly.

Unfortunately, we can't create something like that as an interface like that is more
of a frontend project than something related to Django. However, if you want, it is a
good exercise if you are looking to improve your frontend skills.

For this project, we will create a simple Django form where the user can manually
enter the JSON to define the structure of their forms. We will provide basic
validation and editing capabilities. So let's start by creating our form. Change the
`main/forms.py` file to match the following code:

```python
import json
from django import forms


class NewDynamicFormForm(forms.Form):
    form_pk = forms.CharField(widget=forms.HiddenInput(), required=False)
    title = forms.CharField()
    schema = forms.CharField(widget=forms.Textarea())

    def clean_schema(self):
        schema = self.cleaned_data["schema"]
        try:
            schema = json.loads(schema)
        except:
            raise forms.ValidationError("Invalid JSON. Please submit valid JSON for the schema")

        return schema
```

The form itself is pretty simple. However, we need to take a closer look at the
`clean_schema` method. When the `is_valid()` method is called on a Django form, it
goes through a couple of steps. The exact details can be found in the documentation
at <https://docs.djangoproject.com/en/stable/ref/forms/validation/>.
However, a good approximation is that Django calls two different types of clean
methods on the form class. First, it looks for methods named clean_<fieldname>
for each field defined on the form. This method should return the cleaned value of
the field that will then be stored in the `cleaned_data` dictionary of the form. Next,
Django calls the `clean` method on the form class. This method is meant to clean
data for fields that depend on each other as dependent cleaning should not be done
in individual `clean_<fieldname>` methods. The `clean` method should return a
dictionary that will replace the form's `cleaned_data` dictionary completely.

if the data is not in the expected format. If `ValidationError` is raised in a
`clean_<fieldname>` method, the error is tied to that field and displayed next to it
when the form is rendered. If the error is raised in the general `clean` method, the
error is displayed in the form error list that is displayed right before the form fields
start in the HTML of the form.

As we wish to make sure that the data entered by the user in the schema field is
valid JSON, we use `json.loads` on the user-entered data and raise an exception if
the call fails. One important thing to note is that we return a modified version of the
schema if our `clean_schema` call succeeds. This is because, in our model, we need
to save a Python dictionary instead of a JSON string. Our `JSONField` model field
handles the necessary conversion from a Python object to a JSON string when saving
to the database.

Another side note is that the `clean_schema` method is not given any arguments. It
must get the value for the schema field from the form's cleaned_data dictionary.
With our form created, let's add a view to `main/views.py` to display this form. First,
import our new form and the `json` library at the top:

```python
import json
from main.forms import NewDynamicFormForm
```

Then add the following code to the view:

```python
class CreateEditFormView(FormView):
    form_class = NewDynamicFormForm
    template_name = "create_edit_form.html"

    def get_initial(self):
        if "form_pk" in self.kwargs:
            form = FormSchema.objects.get(pk=self.kwargs["form_pk"])
            initial = {
                "form_pk": form.pk,
                "title": form.title,
                "schema": json.dumps(form.schema)
            }
        else:
            initial = {}

        return initial

    def get_context_data(self, **kwargs):
        ctx = super(CreateEditFormView, self).get_context_data(**kwargs)
        if "form_pk" in self.kwargs:
            ctx["form_pk"] = self.kwargs["form_pk"]

        return ctx

    def form_valid(self, form):
        cleaned_data = form.cleaned_data
        if cleaned_data.get("form_pk"):
            old_form = FormSchema.objects.get(pk=cleaned_data["form_pk"])
            old_form.title = cleaned_data["title"]
            old_form.schema = cleaned_data["schema"]
            old_form.save()
        else:
            new_form = FormSchema(title=cleaned_data["title"],
            schema=cleaned_data["schema"])
            new_form.save()

        return HttpResponseRedirect(reverse("home"))
```

Next, create the `main/templates/create_edit_form.html` file and put this code in
there:

```html
{% extends "base.html" %}

{% block content %}
<h1>Create/Edit Form</h1>

{% if form_pk %}
<form action="{% url 'edit-form' form_pk=form_pk%}" method="post">{% csrf_token %}
{% else %}
<form action="{% url 'create-form' %}" method="post">{% csrf_token %}
{% endif %}

{{ form.as_p }}
<input type="submit" value="Create Form" />
</form>
{% endblock %}
```

Finally, import this new view in our `formmason/urls.py` and add two patterns for it
to the `urlpatterns` file. Here is the final `formmason/urls.py` f﻿ile:

```python
from django.conf.urls import url
from main.views import CreateEditFormView
from main.views import CustomFormView
from main.views import FormResponsesListView
from main.views import HomePageView
urlpatterns = [
    url(r'^$', HomePageView.as_view(), name='home'),
    url(r'^form/(?P<form_pk>\d+)/$', CustomFormView.as_view(),
    name='custom-form'),
    url(r'^form/(?P<form_pk>\d+)/responses/$', FormResponsesListView.
    as_view(), name='form-responses'),
    url(r'form/new/$', CreateEditFormView.as_view(), name='createform'),
    url(r'form/(?P<form_pk>\d+)/edit/$', CreateEditFormView.as_view(),
    name='edit-form'),
]
```

We are using the same view twice—once as a create view and once as an edit view.
If you look at the code for the view, we provide the form with initial data from a
FormSchema object if the `form_pk` keyword argument was matched from the URL.
If it wasn't, we know that we are creating a new form and our initial data dictionary
is empty. One thing to note is that in the initial data, we use `json.dumps(form.schema)` to provide the initial value for the schema field. This is the reverse of what
we do in the `clean_schema` method of our form. That's because `form.schema` is a
Python object; however, our frontend needs to show the JSON for the structure. So
we convert from a Python object to a JSON string using the json.dumps method.

We handle both the cases—new form creation and form editing—in the `form_valid`
method of our view. On successfully saving or creating our form, we redirect the
user to the home page again.

The last thing we need to do is add links to our base template and home page to
create and edit forms. First, add this link right after the link for **Home** in `templates/base.html`:

```html
<a href="{% url 'create-form' %}">Create New Form</a>
```

In `main/templates/home.html`, add this after the link for **See Responses**:

```html
<br /><a href="{% url 'edit-form' form_pk=form.pk %}">Edit Form</a>
```

That's it. Let's test it out. Open the home page at `http://127.0.0.1:8000/` first and
you should see something similar to the following screenshot:

![Form Mason: Available Forms](img/Mason_Forms2.png)

Click on the **Edit Form** button and you should see the following page:

![Form Mason: Create/Edit Form](img/Mason_EditForm.png)

Let's change the title to something new and then type this in the `Schema` field:

```python
{"years_of_experience": "number", "occupation": "string"}
```

Click on the **Create New Form** link. This should save the form and take you back
to the home page. The title of your form on the home page should be changed to
whatever you edited it to. Click on the link to see the form, and you should see a
screen similar to this:

![Form Mason: Custom Form](img/Mason_CustomForm.png)

Nice! Our form has changed to match the schema that we entered while editing
the form. Let's take a look at what our responses page looks like. Submit some test
data with the new form schema and open up the responses page. You should see
something like this:

![Form Mason: Responses for Edited Form](img/Mason_ResponsesForEditedForm.png)

This page works as expected, without having to change any of the code. You'll see
that the previous responses are no longer visible; however, there seems to be some
empty cells above the latest response. We'll look into fixing this and other small
issues next. However, our application is complete except for a few small fixes that we
need to do.

You should use the **Create New Form** link at the top of the pages as well to create a
new form and test it out. It should work as expected.

## Small fixes

I noticed three small errors while working on this last section that I'd like to fix. First
of all, if you take a closer look at the form submission page (where the user can enter
data in a custom form), you'll notice that the heading on the top says **Custom Form**.
This is from our first test, when our form schema was defined in a hardcoded JSON
string and didn't have a title. As our form model now has a title field, edit `main/templates/custom_form.html` and change the `h1` tag to match the following:

```html
<h1>{{ form_schema.title }}</h1>
```

Next, edit `CustomFormView` in `main/views.py` and add the this `get_context_data` method to the class:

```python
def get_context_data(self, **kwargs):
    ctx = super(CustomFormView, self).get_context_data(**kwargs)

    form_schema = FormSchema.objects.get(pk=self.kwargs["form_pk"])
    ctx["form_schema"] = form_schema

    return ctx
```

Look at the form detail page again. This time, the heading should reflect the form
title, as shown in the following screenshot:

![Form Mason: Edited Form](img/Mason_EditForm2.png)

The next error that I noticed was on the form editing page. Right now, whether you
are creating a new form or editing an existing one, the heading always says `Create/Edit Form` and the **Submit** button always says **Create New Form((. Let's modify our
`main/templates/create_edit_form.html` template to tell the user explicitly what
action they are taking. Here is the final code for the template:

```html
{% extends "base.html" %}

{% block content %}
{% if form_pk %}
    <h1>Edit Form</h1>
{% else %}
    <h1>Create Form</h1>
{% endif %}

{% if form_pk %}
<form action="{% url 'edit-form' form_pk=form_pk%}" method="post">{%
csrf_token %}
{% else %}
<form action="{% url 'create-form' %}" method="post">{% csrf_token %}
{% endif %}
    {{ form.as_p }}
    {% if form_pk %}
        <input type="submit" value="Save Form" />
    {% else %}
        <input type="submit" value="Create Form" />
    {% endif %}
</form>
{% endblock %}
```

Now if you edit an existing form or create a new one, the heading and button should
match that action:

![Form Mason: Edit Form](img/Mason_SaveForm.png)

The last issue I saw was on the responses page. If you edit a form that already has
responses and the new form has none of the fields that the old form had, you will
see some empty table rows at the start. That's because even though our view code
doesn't include any data for those rows, it still includes an empty row. Let's fix this
by modifying the `get_context_data` method of `FormResponsesListView` in `main/views.py`. Look for this piece of code:

```python
for field_name in form_fields:
    if field_name in response_data:
        response_values.append(response_data[field_name])
    else:
        response_values.append('')
responses_list.append(response_values)
```

Change it to the following:

```python
for field_name in form_fields:
    if field_name in response_data:
        response_values.append(response_data[field_name])
    else:
        response_values.append('')
if any(response_values):
    responses_list.append(response_values)
```

The last two lines, the highlighted ones, are the changes. The `any` function is a builtin
Python method that returns `True` if any of the values in the given list evaluates
to True. If our `responses_list` contains all empty strings, which is the case if our
new form structure has no fields that overlap with the old form structure, the if
condition will fail and we will not include a completely empty row in our responses
list. Try looking at the responses list page again and you should see that the empty
rows have now disappeared:

![Form Mason: Responses for Edited Form](img/Mason_ResponsesForEditedForm2.png)

That's it. Our last, and most ambitious, application is complete. Congratulations! You
have come a long way from the simple blog that we created at the start of this book.
You should now be ready to tackle any web application project that comes your way.

## Summary

This chapter was more about exploration than new concepts. The only major new
concept that you learned was about the inner workings of Django forms, and this
section took up fewer than five pages. The rest of the chapter was about finding
solutions to problems that we encountered along the way.

As I said at the start of this chapter, this was the most complicated application that
we have created in the book. It s also likely to be one of the more complicated
applications you will create at the start of your career. It is by no means a polished
product; that's more because we didn't do any frontend work, rather than some lack
of features in the Django backend.

While the knowledge we gained about Django forms in this chapter will prove
invaluable to you, I would like the main takeaway from it to be the approach to
solving problems. We found a problem we needed to solve, we experimented in the
Django shell, and we found a solution. Sometimes it's a bit more complicated than
that. You probably need to search the Internet for existing solutions or ask your
colleagues. However, after finding answers that seem to be relevant to your problem,
you will always have to go back to the experimentation and make sure that the
solution really does solve your problem.

The way I showed you how to play around in the Django shell is just one of the
many ways that you can choose to experiment with different solutions. As I've
mentioned before, everyone has their own technique. Some use management
commands, some use simple Python scripts, and some write test code in the views.
In the end, you will come up with a way that you like the best.

Another thing that I hope you take away from this chapter is the use of external
libraries to help solve our problems. When we needed a way to store JSON in our
database, we used the jsonfield library that was available as open source instead
of rolling our own custom solution. This has benefits and drawbacks. The benefits
are that if you find a package, such as `jsonfield` that is widely used, you will get
a solution that has been tested by many before you. Due to that testing, the library
will be much more stable than what you, or any person, can come up with on your
own. The drawback of using something external is that you have no control over
the direction the project takes and in some projects, you'll have to painstakingly go
over the code for the external library to make sure that it meets any regulations your
project might have about external code usage.


However, personally, I prefer to use a tried and tested third-party library every time
over creating something by myself. For me, the pros usually outweigh the cons.


That's it! This was the final chapter in this epic journey that we started with a simple
blog! We have come a long way from where we started; however, I'm sure that your
journey as a Django web developer is just starting. You have many more exciting
things to look forward to, and I wish you the best of luck in this journey. I hope you
enjoyed reading this book as much as I enjoyed writing it. Bon voyage!